In [1]:
# Dependencies
import json
import requests
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
# Specify the URL
request_url = 'https://api.usa.gov/crime/fbi/sapi/api/data/hatecrime/national/BIAS_INCIDENT?API_KEY=qg50lz2p4PFyRNeEtptg63BHzgpf2tNB3gwnpJhM'

response = requests.get(request_url)
if response.status_code == 200:
    print('code 200 == woohoo!')

code 200 == woohoo!


In [3]:
crime_data = response.json()
print(json.dumps(crime_data, indent=2, sort_keys=True))

{
  "pagination": {
    "count": 24,
    "page": 0,
    "pages": 1,
    "per_page": 0
  },
  "results": [
    {
      "anti_american_indian": 11,
      "anti_arab": 0,
      "anti_asian": 17,
      "anti_atheism": 1,
      "anti_bisexual": 12,
      "anti_black": 330,
      "anti_buddhist": 0,
      "anti_catholic": 11,
      "anti_disability": 10,
      "anti_eastern_orthodox": 0,
      "anti_female": 0,
      "anti_gay_male": 77,
      "anti_gay_mixed": 47,
      "anti_gender_non_conforming": 0,
      "anti_hawaiian": 0,
      "anti_heterosexual": 1,
      "anti_hindu": 0,
      "anti_hispanic": 80,
      "anti_islamic": 0,
      "anti_jehovah": 0,
      "anti_jewish": 42,
      "anti_lesbian": 26,
      "anti_male": 0,
      "anti_morman": 0,
      "anti_multiple_race": 24,
      "anti_multiple_religions": 18,
      "anti_not_hispanic": 28,
      "anti_other_christian": 0,
      "anti_other_religion": 18,
      "anti_physical_disability": 5,
      "anti_protestant": 6,
      "anti_s

In [6]:
allresults = crime_data['results']

df = pd.DataFrame(allresults)
df

,anti_atheism,anti_morman,anti_other_christian,anti_male,anti_buddhist,anti_sikh,anti_multiple_race,anti_hawaiian,anti_disability,anti_arab,...,anti_other_religion,anti_gay_male,anti_eastern_orthodox,anti_jewish,anti_gender_non_conforming,anti_female,anti_catholic,anti_not_hispanic,multiple_bias,data_year
0,1,0,0,0,0,0,24,0,10,0,...,18,77,0,42,0,0,11,28,0,2010
1,0,0,0,0,0,0,13,0,1,0,...,10,39,0,36,0,0,2,19,0,2006
2,0,0,0,0,0,0,4,0,4,1,...,5,17,0,8,0,0,1,3,0,2000
3,0,0,0,0,0,0,4,0,0,1,...,3,28,0,18,0,0,2,13,0,2003
4,0,0,0,0,0,0,1,0,0,0,...,2,11,0,3,0,0,0,3,0,1999
5,1,0,0,0,0,0,10,0,2,0,...,4,38,0,15,0,0,5,14,0,2005
6,0,0,0,0,0,0,0,0,0,0,...,0,2,0,1,0,0,0,0,0,1996
7,3,0,0,3,0,0,28,1,22,0,...,22,116,0,51,1,3,8,96,0,2013
8,1,0,0,0,0,0,17,0,5,0,...,12,73,0,68,0,0,6,29,0,2009
9,0,0,0,0,0,0,16,0,5,0,...,7,16,0,24,0,0,4,9,0,2004
